In [1]:
#Libraries
import pandas as pd                  # Data Manipulation
import numpy as np                   # Matrices support
import matplotlib.pyplot as plt      # Plotting
import seaborn as sns                # Statistical Visualization
from matplotlib import cm
from scipy import stats
from numpy.polynomial.polynomial import polyfit
from random import randint

In [2]:
data = pd.read_excel('201713_ECOAS_Anonimizado.xlsx')

In [3]:
data.head(5)

,Ejercicio,Crn,ID_Prof,Rectoría del Campus,Campus,División,Departamento,Tesis en título,Número de Grupo,Grupo Virtual,...,Conference Proceeding,Journal Q1/Q2,Journal Q3/Q4,Total Docs,Book Norm.,Book Series Norm.,Conference Proceeding Norm.,Journal Q1/Q2 Norm.,Journal Q3/Q4 Norm.,Total Docs. Norm.
0,201713,300,1453688,Región Sur,Campus Toluca,"69 - Arq, Arte y Diseño",DII - Diseño Industrial,NaN,1,NO,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,201713,317,701591,Región Sur,Campus Toluca,"69 - Arq, Arte y Diseño",AR - Arquitectura,NaN,1,NO,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,201713,320,1089805,Región Sur,Campus Toluca,"69 - Arq, Arte y Diseño",AR - Arquitectura,NaN,2,NO,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,201713,321,925702,Región Sur,Campus Toluca,"69 - Arq, Arte y Diseño",DII - Diseño Industrial,NaN,1,NO,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,201713,322,925702,Región Sur,Campus Toluca,"69 - Arq, Arte y Diseño",DII - Diseño Industrial,NaN,2,NO,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


- Hacer una variable de clasificación (bueno/malo) para cada profesor. Debemos de generar un criterio de que es considerado bueno, en base a las 3 respuestas de la ECOA.
- Encontrar que variables son las que impactan más a las respuestas de la ECOA.
- Una vez que conozco las variables que predicen mejor el performance. Aplicar clustering para ver si los individuos poseen características similares, respecto a lo deseado.

In [4]:
#La variable 'Crn UV' únicamente tiene el valor "No aplica", por lo que descartamos esta columna
data = data.drop(['Crn UV'],axis=1)

In [5]:
#Cuenta la cantidad de profesores por nacionalidad y agrega una columna del porcentaje del total
prof_nacionalidad =  data[['Nacionalidad','ID_Prof']].groupby(['Nacionalidad'],as_index=False).count().sort_values(['ID_Prof'],ascending=False)
prof_nacionalidad['porcentaje'] = prof_nacionalidad['ID_Prof']/len(data)*100
prof_nacionalidad.head()

,Nacionalidad,ID_Prof,porcentaje
34,Mexicana,14866,94.201888
21,Estadounidense,122,0.773082
20,Española,88,0.557633
30,Italiana,74,0.468918
11,Colombiana,68,0.430898


Debido a que hay 56 nacionalidad diferentes de profesores, creamos una variable llamada mxn_sn (mexicano si o no) que es 1 es mexicano y 0 cuando es extranjero

In [6]:
a = [];
for index,value in enumerate(data['Nacionalidad']):
    if value == 'Mexicana':
        a.append(1);
    else:
        a.append(0);
data['mxn_sn'] = a        

In [7]:
prof_genero = data[['Género','ID_Prof']].groupby(['Género'],as_index=False).count()
prof_genero['porcentaje'] = prof_genero['ID_Prof']/len(data)*100
prof_genero

,Género,ID_Prof,porcentaje
0,Femenino,6243,39.560231
1,Masculino,9538,60.439769


In [8]:
#from sklearn.cluster import KMeans
c1 = data['Campus']=='Campus Monterrey';
c2 = data['División'] == 'IN - Esc. Ingeniería y Ciencias'
data[c1&c2].drop(['Campus','División','Crn','ID_Prof','Rectoría del Campus','Ejercicio',],axis=1)

,Departamento,Tesis en título,Número de Grupo,Grupo Virtual,Categoría Laboral,Puesto (Escolar),Puesto (RH),Nacionalidad,Género,Edad al semestre,...,Journal Q1/Q2,Journal Q3/Q4,Total Docs,Book Norm.,Book Series Norm.,Conference Proceeding Norm.,Journal Q1/Q2 Norm.,Journal Q3/Q4 Norm.,Total Docs. Norm.,mxn_sn
5874,BIN - Bioingeniería,NaN,4,NO,-,Profesor cátedra externo,Profesor cátedra,Mexicana,Masculino,34.162902,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5875,BIN - Bioingeniería,NaN,5,NO,-,Profesor cátedra externo,Profesor cátedra,Mexicana,Masculino,34.162902,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5876,PC - Ciencias,NaN,4,NO,PLA - Planta,Profesor planta,Profesor,Mexicana,Masculino,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5877,PC - Ciencias,NaN,5,NO,PLA - Planta,Profesor planta,Profesor,Mexicana,Masculino,59.148528,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5878,PC - Ciencias,NaN,6,NO,PLA - Planta,Profesor planta directivo,Director Asociado de Departamento Region,Mexicana,Masculino,58.762491,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5879,PC - Ciencias,NaN,7,NO,PLA - Planta,Profesor planta directivo,Director Asociado de Departamento Region,Mexicana,Masculino,58.762491,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5883,PC - Ciencias,NaN,16,NO,-,Profesor planta directivo,Director de Departamento Regional,Mexicana,Masculino,43.208761,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5884,PC - Ciencias,NaN,4,NO,000 - Indefinido,Profesor cátedra externo,Profesor cátedra,Mexicana,Masculino,58.179329,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5885,PC - Ciencias,NaN,3,NO,000 - Indefinido,Profesor cátedra externo,Profesor cátedra,Mexicana,Masculino,58.179329,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
5887,PC - Ciencias,NaN,5,NO,000 - Indefinido,Profesor cátedra externo,Profesor cátedra,Mexicana,Masculino,49.514031,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
